In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### First step is to pre-process the CSV files
For that import CSV and os Module

In [ ]:
import csv,os

### User defined function data_process to process the CSV files.
If the CSV files are changed then the code needs to be changed

In [ ]:
def data_process(file_path):
  file_name = os.path.basename(file_path)
  print(file_name)
  repaired_file_name = "repaired_" + file_name
  with open(file_path, 'r', encoding = 'utf-8') as infile, open(repaired_file_name,'w', encoding='utf-8') as outfile:
    for line in infile.readlines():
        error = 0
        try:
          # first read the csv file as a normal file to remove some special characters as string ","
            line = line.replace('","', '')
            # line = line.replace('",', '') # '",'    I am not able to remove this 2 character as a string, because it breaks the format of the 
            # csv file. But it can be done easily using string slicing . But my target is to keep it simple.
            outfile.write(line)
        except:
            error += 1
        
    if error==0:
      print('all lines are written successfully')
    else:
      print(f"missed {error} lines")

  # after cleaning and storing data on a new csv file, now is the task is to extract the information from new csv file
  with open(repaired_file_name, 'r', encoding = 'utf-8-sig') as file:
      text1 = []                  # to store sentence for 1st sentence column 
      text2 = []                  # to store sentence for 2nd sentence column 
      label = []                  # to store the labels
      reader = csv.reader(file)
      count=1
      error = 0
      for row in reader:
          first_element = row[0]
          # check.append(first_element)
          try:
              sent2 = first_element.split('\t')[6]
              sent1 = first_element.split('\t')[5]
              target = first_element.split('\t')[0]
              
              text2.append(sent2)
              text1.append(sent1)
              label.append(target)
          except:
              error+=1 
      if error==0:
        print('all lines are written successfully')
      else:
        print(f"missed {error} lines")
  return {'similarity': label,'text1': text1, 'text2': text2}

## Set path of the dataset

In [ ]:
# Just give the file path
trainig_data = data_process('/content/drive/MyDrive/delhi iit/stsbenchmark/sts-train.csv')
type(trainig_data)
test_data = data_process('/content/drive/MyDrive/delhi iit/stsbenchmark/sts-test.csv')
type(test_data)
val_data = data_process('/content/drive/MyDrive/delhi iit/stsbenchmark/sts-dev.csv')
type(val_data)

sts-train.csv
all lines are written successfully
missed 210 lines
sts-test.csv
all lines are written successfully
missed 180 lines
sts-dev.csv
all lines are written successfully
missed 86 lines


dict

# Convert CSV to Pandas dataframe

In [ ]:
import pandas as pd
# train data
train_df = pd.DataFrame(trainig_data)
train_df.head()

,similarity,text1,text2
0,main-captions,A plane is taking off.,An air plane is taking off.
1,main-captions,A man is playing a large flute.,A man is playing a flute.
2,main-captions,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,Three men are playing chess.,Two men are playing chess.
4,main-captions,A man is playing the cello.,A man seated is playing the cello.


In [ ]:
# Test data
test_df = pd.DataFrame(test_data)
test_df.head()

,similarity,text1,text2
0,main-captions,A girl is styling her hair.,A girl is brushing her hair.
1,main-captions,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.
2,main-captions,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.
3,main-captions,A man is cutting up a cucumber.,A man is slicing a cucumber.
4,main-captions,A man is playing a harp.,A man is playing a keyboard.


In [ ]:
# validation set
val_df = pd.DataFrame(val_data)
val_df.head()

,similarity,text1,text2
0,main-captions,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.
1,main-captions,A young child is riding a horse.,A child is riding a horse.
2,main-captions,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.
3,main-captions,A woman is playing the guitar.,A man is playing guitar.
4,main-captions,A woman is playing the flute.,A man is playing a flute.


In [ ]:
# Shape of the data
print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {val_df.shape[0]}")
print(f"Total test samples: {test_df.shape[0]}")

Total train samples : 5005
Total validation samples: 1230
Total test samples: 1093


## Setup

Note: install HuggingFace `transformers` via `pip install transformers`

In [ ]:
# pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

In [ ]:
train_df['similarity'].unique()

array(['main-captions', 'main-forum', 'main-news'], dtype=object)

In [ ]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 2

# Labels in our dataset.
labels = list(train_df['similarity'].unique())
labels

['main-captions', 'main-forum', 'main-news']

Let's look at one sample from the dataset:

In [ ]:
print(f"Sentence1: {train_df.loc[1, 'text1']}")
print(f"Sentence2: {train_df.loc[1, 'text2']}")
print(f"Similarity: {train_df.loc[1, 'similarity']}")

Sentence1: A man is playing a large flute.
Sentence2: A man is playing a flute.
Similarity: main-captions


## Preprocessing

In [ ]:
# We have some NaN entries in our train data, we will simply drop them.
print("Number of missing values")
print(train_df.isnull().sum())
train_df.dropna(axis=0, inplace=True)

Number of missing values
similarity    0
text1         0
text2         0
dtype: int64


Distribution of our training targets.

In [ ]:
print("Train Target Distribution")
print(train_df.similarity.value_counts())

Train Target Distribution
main-news        2577
main-captions    1994
main-forum        434
Name: similarity, dtype: int64


Distribution of our validation targets.

In [ ]:
print("Validation Target Distribution")
print(val_df.similarity.value_counts())

Validation Target Distribution
main-captions    623
main-forums      331
main-news        276
Name: similarity, dtype: int64


In [ ]:
train_df = (
    train_df[train_df.similarity != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)
valid_df = (
    val_df[val_df.similarity != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)

One-hot encode training, validation, and test labels.

In [ ]:
train_df["label"] = train_df["similarity"].apply(
    lambda x: 0 if x == "main-captions" else 1 if x == "main-forum" else 2
)
y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=3)

valid_df["label"] = valid_df["similarity"].apply(
    lambda x: 0 if x == "main-captions" else 1 if x == "main-forum" else 2
)
y_val = tf.keras.utils.to_categorical(valid_df.label, num_classes=3)

test_df["label"] = test_df["similarity"].apply(
    lambda x: 0 if x == "main-captions" else 1 if x == "main-forum" else 2
)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=3)

In [ ]:
train_df.label.unique()

array([2, 0, 1])

## Keras Custom Data Generator
Ref: Keras.io

In [ ]:

class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


## Build the model.
Ref: Keras.io

In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7ffab14ab1d0>
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',      

Create train and validation data generators

In [ ]:
train_data = BertSemanticDataGenerator(
    train_df[["text1", "text2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[["text1", "text2"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

## Train the Model

Training is done only for the top layers to perform "feature extraction",
which will allow the model to use the representations of the pretrained model.

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/2
156/156 [==============================] - ETA: 0s - loss: 0.0942 - acc: 0.9659 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


156/156 [==============================] - 2819s 18s/step - loss: 0.0942 - acc: 0.9659 - val_loss: 0.3828 - val_acc: 0.8215
Epoch 2/2
156/156 [==============================] - 2627s 17s/step - loss: 0.0350 - acc: 0.9894 - val_loss: 0.7724 - val_acc: 0.7541


## Fine-tuning

This step must only be performed after the feature extraction model has
been trained to convergence on the new data.

This is an optional last step where `bert_model` is unfreezed and retrained
with a very low learning rate. This can deliver meaningful improvement by
incrementally adapting the pretrained features to the new data.

In [ ]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_masks[0][0]',  

# Train the entire model end-to-end.

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Epoch 1/2


156/156 [==============================] - 6849s 44s/step - loss: 0.0208 - accuracy: 0.9930 - val_loss: 0.4635 - val_accuracy: 0.8355
Epoch 2/2
156/156 [==============================] - 7004s 45s/step - loss: 0.0065 - accuracy: 0.9978 - val_loss: 0.9686 - val_accuracy: 0.7549


## Evaluate model on the test set

In [ ]:
test_data = BertSemanticDataGenerator(
    test_df[["text1", "text2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


34/34 [==============================] - 439s 13s/step - loss: 1.2315 - accuracy: 0.8180


[1.2314528226852417, 0.8180146813392639]

# Save model

In [ ]:
model.save("with fine tunnig.h5")

## Inference on custom sentences

In [ ]:

def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba


Check results on some example sentence pairs.

In [ ]:
sentence1 = "Two women are observing something together."
sentence2 = "Two women are standing with their eyes closed."
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 7s 7s/step


('main-captions', ' 1.00%')

Check results on some example sentence pairs.

In [ ]:
sentence1 = "A smiling costumed woman is holding an umbrella"
sentence2 = "A happy woman in a fairy costume holds an umbrella"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 577ms/step


('main-captions', ' 1.00%')

Check results on some example sentence pairs

In [ ]:
sentence1 = "A soccer game with multiple males playing"
sentence2 = "Some men are playing a sport"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/1 [==============================] - 1s 575ms/step


('main-captions', ' 1.00%')